<a href="https://colab.research.google.com/github/paddy3696/CNN_Pytorch/blob/main/Inat_cnn_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Apr 17 20:01:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt

In [3]:
%pip install wandb -q

# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

# WandB – Import the wandb library
import wandb
#wandb.init(project='mnist_classification', entity='paddy3696')

     |████████████████████████████████| 2.1MB 13.6MB/s 
     |████████████████████████████████| 163kB 51.0MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 133kB 52.4MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 


In [4]:
train_gpu = torch.cuda.is_available()

if not train_gpu:
    print('CUDA is not available. Training on CPU')
else:
    print('CUDA is available! Training on GPU')

CUDA is available! Training on GPU


In [5]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=4bd717d72ddc962b3c59c7409bd683d0b7ea85ed4e83aa2c37c9a3b8ac4ba48c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget
wget.download('https://storage.googleapis.com/wandb_datasets/nature_12K.zip')
!unzip /content/nature_12K.zip

In [7]:
class MyNN(nn.Module):
  def __init__(self, drop_out = 0.2, batch_norm = 'Yes', filter_n = 32,  filter_org = 'same',hidden_out = 128):

    super(MyNN,self).__init__()
    
    self.drop_out = drop_out
    self.batch_norm = batch_norm
    self.filter_n = filter_n
    self.filter_org = filter_org
    self.hidden_out = hidden_out
    
    if self.filter_org == 'same':
      self.conv_1 = nn.Conv2d(3,self.filter_n,kernel_size=3,padding=1)
      self.conv_1_bn = nn.BatchNorm2d(self.filter_n)
      self.conv_2 = nn.Conv2d(self.filter_n,self.filter_n,kernel_size=3,padding=1)
      self.conv_2_bn = nn.BatchNorm2d(self.filter_n)
      self.conv_3 = nn.Conv2d(self.filter_n,self.filter_n,kernel_size=3,padding=1)
      self.conv_3_bn = nn.BatchNorm2d(self.filter_n)
      self.conv_4 = nn.Conv2d(self.filter_n,self.filter_n,kernel_size=3,padding=1)
      self.conv_4_bn = nn.BatchNorm2d(self.filter_n)
      self.conv_5 = nn.Conv2d(self.filter_n,self.filter_n,kernel_size=3,padding=1)
      self.conv_5_bn = nn.BatchNorm2d(self.filter_n)

      self.fc1 = nn.Linear(self.filter_n*7*7,self.hidden_out)
      self.fc2 = nn.Linear(self.hidden_out,10)

    if self.filter_org == 'double_up':
      self.conv_1 = nn.Conv2d(3,self.filter_n,kernel_size=3,padding=1)
      self.conv_1_bn = nn.BatchNorm2d(self.filter_n)
      self.conv_2 = nn.Conv2d(self.filter_n,self.filter_n*2,kernel_size=3,padding=1)
      self.conv_2_bn = nn.BatchNorm2d(self.filter_n*2)
      self.conv_3 = nn.Conv2d(self.filter_n*2,self.filter_n*4,kernel_size=3,padding=1)
      self.conv_3_bn = nn.BatchNorm2d(self.filter_n*4)
      self.conv_4 = nn.Conv2d(self.filter_n*4,self.filter_n*8,kernel_size=3,padding=1)
      self.conv_4_bn = nn.BatchNorm2d(self.filter_n*8)
      self.conv_5 = nn.Conv2d(self.filter_n*8,self.filter_n*16,kernel_size=3,padding=1)
      self.conv_5_bn = nn.BatchNorm2d(self.filter_n*16)

      self.fc1 = nn.Linear(self.filter_n*16*7*7,self.hidden_out)
      self.fc2 = nn.Linear(self.hidden_out,10)

    if self.filter_org == 'double_down':
      self.conv_1 = nn.Conv2d(3,self.filter_n,kernel_size=3,padding=1)
      self.conv_1_bn = nn.BatchNorm2d(self.filter_n)
      self.conv_2 = nn.Conv2d(self.filter_n, int(self.filter_n/2),kernel_size=3,padding=1)
      self.conv_2_bn = nn.BatchNorm2d(int(self.filter_n/2))
      self.conv_3 = nn.Conv2d(int(self.filter_n/2), int(self.filter_n/4),kernel_size=3,padding=1)
      self.conv_3_bn = nn.BatchNorm2d(int(self.filter_n/4))
      self.conv_4 = nn.Conv2d(int(self.filter_n/4), int(self.filter_n/8),kernel_size=3,padding=1)
      self.conv_4_bn = nn.BatchNorm2d(int(self.filter_n/8))
      self.conv_5 = nn.Conv2d(int(self.filter_n/8), int(self.filter_n/16),kernel_size=3,padding=1)
      self.conv_5_bn = nn.BatchNorm2d(int(self.filter_n/16))

      self.fc1 = nn.Linear(int(self.filter_n/16)*7*7,self.hidden_out)
      self.fc2 = nn.Linear(self.hidden_out,10)

    self.pool = nn.MaxPool2d(2,2)
    self.dropout = nn.Dropout(self.drop_out)

  def forward(self,x):
    if self.batch_norm == 'Yes':
      x = self.pool(F.relu(self.conv_1_bn(self.conv_1(x))))
      x = self.pool(F.relu(self.conv_2_bn(self.conv_2(x))))
      x = self.pool(F.relu(self.conv_3_bn(self.conv_3(x))))
      x = self.pool(F.relu(self.conv_4_bn(self.conv_4(x))))
      x = self.pool(F.relu(self.conv_5_bn(self.conv_5(x))))
      if self.filter_org == 'same':
        x = x.view(-1,self.filter_n*7*7)
      elif self.filter_org == 'double_up':
        x = x.view(-1,self.filter_n*16*7*7)
      elif self.filter_org == 'double_down':
        x = x.view(-1,int(self.filter_n/16)*7*7)
      x = self.dropout(x)
      x = F.relu(self.fc1(x))
      x = self.dropout(x)
      x = self.fc2(x)

    if self.batch_norm == 'No':
      x = self.pool(F.relu(self.conv_1(x)))
      x = self.pool(F.relu(self.conv_2(x)))
      x = self.pool(F.relu(self.conv_3(x)))
      x = self.pool(F.relu(self.conv_4(x)))
      x = self.pool(F.relu(self.conv_5(x)))
      if self.filter_org == 'same':
        x = x.view(-1,self.filter_n*7*7)
      elif self.filter_org == 'double_up':
        x = x.view(-1,self.filter_n*16*7*7)
      elif self.filter_org == 'double_down':
        x = x.view(-1,int(self.filter_n/16)*7*7)
      x = self.dropout(x)
      x = F.relu(self.fc1(x))
      x = self.dropout(x)
      x = self.fc2(x)

    #return F.log_softmax(x,dim= 1)
    return x

In [8]:
def fit(model,train_loader,val_loader,optimizer,epoch,criterion):
  acc_epoch = []

  for epoch in range(1, epoch+1):
    train_loss = 0.0
    val_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    print('training started')
    model.train()
    for data, target in train_loader:
      if train_gpu:
        data, target = data.cuda(), target.cuda()

      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()*data.size(0)

    train_loss = train_loss/len(train_loader.sampler)
    print('Training over')
    model.eval()
    for data, target in val_loader:
      batch_data_size = data.size(0)
      if train_gpu:
        data, target = data.cuda(), target.cuda()
      output = model(data)
      loss = criterion(output, target)
      val_loss += loss.item()*data.size(0)

      _, pred = torch.max(output, 1)    
      correct_tensor = pred.eq(target.data.view_as(pred))
      correct = np.squeeze(correct_tensor.numpy()) if not train_gpu else np.squeeze(correct_tensor.cpu().numpy())

      for i in range(batch_data_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
    
    val_loss = val_loss/len(val_loader.sampler)
    val_accuracy = 100. * np.sum(class_correct) / np.sum(class_total)
    acc_epoch.append(val_accuracy)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, val_loss))
    print('Val Accuracy (Overall): %.2f%% (%2d/%2d)\n' % (val_accuracy,np.sum(class_correct), np.sum(class_total)))

    wandb.log({"Train Loss": train_loss,"Val Loss": val_loss,"Val Accuracy": val_accuracy})
  
  acc_best = max(acc_epoch)
  wandb.log({'accuracy' : acc_best})

In [9]:
sweep_config = {
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'drop_out': {
            'values': [0.2, 0.3]
        },
        'batch_norm': {
            'values': ['Yes', 'No']
        },
        'filter_n': {
            'values': [64, 128]
        
        },
        'batch_size': {
            'values': [16, 32]
        },
        'filter_org': {
            'values': ['same', 'double_up', 'double_down']
        },
        'epoch': {
            'values': [5,10]
        },
        'data_aug': {
            'values': ['Yes', 'No']
        },
        'optimizer': {
            'values': ['SGD','ADAM'] 
        },
        'lr': {
            'values': [0.1,0.01] 
        },
        'hidden_out': {
            'values': [128,196] 
        },
    }
}

In [10]:
#sweep_id = wandb.sweep(sweep_config, entity=#####, project="cnn_inat")

In [11]:
def train():
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'epoch': 10,
      'batch_norm': 'Yes',
      'filter_n': 64,
      'filter_org': 'same',
      'data_aug': 'Yes',
      'optimizer': 'SGD',
      'batch_size': 16,
      'seed': 9,
      'lr': 9,
      'hidden_out':128,
      'drop_out': 0.2,
  }

  # Initialize a new wandb run
  wandb.init(config=config_defaults)
   
  # Config is a variable that holds and saves hyperparameters and inputs
  config = wandb.config

  data_dir = '/content/inaturalist_12K'
                              
  if config.data_aug == 'Yes':
    train_transforms = transforms.Compose([transforms.RandomRotation(30),transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),transforms.ToTensor(), 
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
                                           
  elif config.data_aug == 'No':
    train_transforms = transforms.Compose([transforms.RandomResizedCrop(224), transforms.ToTensor()])

  train_data = datasets.ImageFolder(data_dir + '/train',transform=train_transforms)

  batch_size = config.batch_size
  validation_split = .1
  shuffle_dataset = True
  random_seed= config.seed

  dataset_size = len(train_data)
  indices = list(range(dataset_size))
  split = int(np.floor(validation_split * dataset_size))
  if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
  train_indices, val_indices = indices[split:], indices[:split]

  train_sampler = SubsetRandomSampler(train_indices)
  valid_sampler = SubsetRandomSampler(val_indices)

  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=train_sampler)
  val_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=valid_sampler)
  print('data ready')
  
  model = MyNN(drop_out = config.drop_out, batch_norm = config.batch_norm, filter_n = config.filter_n,  filter_org = config.filter_org, hidden_out =config.hidden_out)
  print(model)

  if train_gpu:
    model.cuda()

  criterion = nn.CrossEntropyLoss()
  #criterion = nn.NLLLoss()
  epoch = config.epoch

  if config.optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=config.lr,momentum=0.1)
  elif config.optimizer == 'ADAM':
    optimizer = optim.Adam(model.parameters(), lr=config.lr)

  wandb.run.name = str(config.filter_n) +'_'+ str(config.filter_org) + '_bs_' + str(config.batch_size)

  fit(model,train_loader,val_loader,optimizer,epoch,criterion)

In [12]:
#wandb.agent(sweep_id, train,count=100)
wandb.agent("22yfvs8p", entity="paddy3696",project="cnn_inat", function =train,count=100)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Agent Starting Run: roezs950 with config:
wandb: 	batch_norm: Yes
wandb: 	batch_size: 32
wandb: 	data_aug: No
wandb: 	drop_out: 0.2
wandb: 	epoch: 10
wandb: 	filter_n: 64
wandb: 	filter_org: double_down
wandb: 	hidden_out: 196
wandb: 	lr: 0.01
wandb: 	optimizer: SGD
wandb: Currently logged in as: paddy3696 (use `wandb login --relogin` to force relogin)


data ready
MyNN(
  (conv_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_3): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_4): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_4_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_5): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_5_bn): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=196, out_features=196, bias=True)
  (fc2): Linear(in_features=196, out_features=10, bias=True)
 

wandb: Ctrl + C detected. Stopping sweep.
